# **Feature Bagging for Outlier Detection**

El algoritmo de **Feature Bagging** es una técnica de aprendizaje automático que combina múltiples modelos para mejorar la precisión y robustez de las predicciones. A diferencia del **Bagging** tradicional, que se basa en la creación de subconjuntos aleatorios de datos, el **Feature Bagging** se centra en la selección aleatoria de características (*features*) para entrenar cada modelo.


De aqui se basa el **Feature Bagging** para la detección de anomalías o *outliers*, que utiliza múltiples modelos entrenados en subconjuntos aleatorios de características o *features* . Dado que muchas técnicas de detección de valores atípicos que calculan distancias dimensionales completas también son de naturaleza local, son sensibles a la selección de características utilizadas en el cálculo de distancias. Además, la presencia de características ruidosas e irrelevantes puede reducir significativamente el rendimiento de la detección de valores atípicos , el ensablaje de múltiples modelos entrenados en subconjuntos aleatorios de características puede ayudar a mitigar este problema , ademas , brindando dos formas de agregación de los factores de decisión mas acorde al problema

## Implementación
La implementación realizada en *PyOD_ADGE* , se baso unicamente en el uso del algoritmo de *Local Outlier Factor* (*LOF*) como modelo base, pero , se debe destacar que eso no impide que se pueda extender a otros algoritmos de detección de anomalías. El algoritmo de *LOF* es un método basado en la densidad que mide la densidad local de un punto en comparación con sus vecinos cercanos. Se utiliza comúnmente para detectar puntos atípicos en conjuntos de datos multidimensionales. 

El modelo base , tiene tal cual sus tres modos :
- ```classic``` : sigue el método propuesto en el paper original
- ```optimize``` : método clasico , pero , optimizando la busqueda de *k-neighbors* paralizando el proceso
- ```sklearn``` : utiliza el modelo de *sklearn* para la detección de anomalías ```sklearn.neighbors.LocalOutlierFactor```

### Hiperparámetros
| Parámetro         | Tipo                  | Valor por defecto | Descripción                                                                                               |
|-------------------|-----------------------|-------------------|-----------------------------------------------------------------------------------------------------------|
| `base_estimator`  | `str`                 | `'classic'`       | Tipo de implementació­n de LOF a usar:  `classic` = propia versión,  `sklearn` = la de scikit-learn.     |
| `n_estimators`    | `int`                 | `10`              | Número de “sub-modelos” (estimadores) a entrenar sobre distintos subconjuntos de features.                 |
| `contamination`   | `float`               | `0.1`             | Proporción esperada de outliers en los datos (entre 0 y 0.5).                                             |
| `combine`         | `str`                 | `'breadth'`       | Método para combinar los scores de los estimadores:  `'breadth'` (breadth-first ranking) o `'Cumulative'`.|
| `n_jobs`          | `int`                 | `2`               | Número de procesos paralelos para el entrenamiento (`joblib.Parallel`).                                   |
| `random_state`    | `int` or `None`       | `None`            | Semilla inicial para el generador de números aleatorios; asegura reproducibilidad.                       |
| `n_neighbors`     | `int`                 | `20`              | Parámetro *k* de LOF: número de vecinos a considerar para calcular la densidad local / outlier score.     |
| `metric`          | `str`                 | `'euclidean'`     | Métrica de distancia a usar en LOF (p.ej. `'euclidean'`, `'manhattan'`, etc.).                            |
| `metric_params`   | `dict`                | `{}`              | Parámetros adicionales específicos de la métrica (e.g. `p` para `'minkowski'`, etc.).                     |


## Demo 
Para mostrar el funcionamiento del algoritmo , se usara el dataset [Dataset de Credit Card Fraud de Kaggle](https://www.kaggle.com/datasets/dhanushnarayananr/credit-card-fraud) , que contiene transacciones de tarjetas de crédito, donde la mayoría son transacciones legítimas y una pequeña fracción son fraudulentas. El objetivo es detectar estas transacciones fraudulentas (outliers) en función de las características de las transacciones.

### Cargar librerías necesarias


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import sys, os

proj_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)
    
from PyOD_ADGE.models.feature_bagging import FeatureBagging

### Cargando el dataset
Se tomara un muestra estratificada de 10000 filas del dataset, pero se puede usar el dataset completo sin problemas, ya que el algoritmo es escalable y no tiene problemas de memoria. 

In [2]:
url_data='https://drive.google.com/uc?export=download&id=1jGJ-Bam7dR9vH4Y6-677LNNpW-PulLfL'
data=pd.read_csv(url_data,header=0)
data.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


In [3]:
data.shape

(1000000, 8)

In [4]:
data['fraud']=data['fraud'].astype(np.int8)
data['fraud'].value_counts()

fraud
0    912597
1     87403
Name: count, dtype: int64

In [5]:
X_f=data.drop(['fraud'],axis=1)
y_f=data['fraud']
X,_,y,_= train_test_split(X_f,y_f,stratify=y_f,train_size=500,random_state=42)

In [6]:
y.value_counts()

fraud
0    456
1     44
Name: count, dtype: int64

In [7]:
X

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order
416902,10.930126,1.596463,2.121041,1.0,0.0,0.0,1.0
766025,8.344195,0.069099,0.724766,1.0,0.0,0.0,1.0
9934,33.796345,0.091538,0.733712,1.0,0.0,0.0,0.0
695805,5.049464,8.951986,1.074492,1.0,1.0,1.0,0.0
933740,3.488985,0.384012,7.482144,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
916205,27.465475,0.068865,1.013117,1.0,0.0,0.0,1.0
560609,14.011734,17.125131,1.319054,1.0,1.0,0.0,1.0
133072,3.647332,0.126717,50.627553,1.0,0.0,0.0,1.0
678092,19.852381,1.752303,5.196821,1.0,0.0,0.0,1.0


### Instanciar el modelo y entrenar
Al instanciar el modelo , el método ```fit``` se encargara de realizar el ajuste de los modelos base y el ajuste de los hiperparámetros. 


In [8]:
X.shape

(500, 7)

In [9]:
model=FeatureBagging(n_estimators=5,n_jobs=-1,random_state=42)
model.fit(X)

In [25]:
model.outlier_factor_[:13]

array([0.98007693, 0.98309461, 0.98432074, 1.00485978, 1.01045237,
       0.98954683, 0.97228137, 1.00324737, 1.01615808, 0.99347962,
       1.06310951, 0.9902954 , 0.98601123])

In [24]:
y[:13]

416902    0
766025    0
9934      0
695805    0
933740    1
546656    0
979136    0
180086    0
488057    0
27937     0
602248    0
411375    1
719874    0
Name: fraud, dtype: int8